# A simple example of parallel computing with Dask on ARM data

**Dask** is a great tool for taking a problem an distrubuting across many cores. This is a **simple example** as in you really could just do this in series. but it neatly shows how easy it is to map a problem to a cluster.  

Now is a great time to learn how to use Dask as there is massive uptake in the community including NSF/NASA funded [Pangeo](https://pangeo.io/)

In [ ]:
#imports
import os # Code for dealing with the file system
import numpy as np
from matplotlib import pyplot as plt
from dask_kubernetes import KubeCluster
from dask.distributed import Client
from dask.distributed import wait, progress

import xarray as xr
%matplotlib inline

First few cells deal with reading data into an X-Array object and work on ONE data file

**For this workshop** We are using Pangeo's Kube cluster set up... I will explain a little in the course...

In [ ]:
# Select 10 'workers' under 'manual scaling' menu below and click 'Scale'
# Click on the 'Dashboard link' to monitor calculation progress
cluster = KubeCluster(n_workers=10, port=33949)
cluster

In [ ]:
client = Client(cluster)

In [ ]:
client

![](pangeo_tech_1.png)

from: http://pangeo.io/architecture.html

In [ ]:
in_directory = os.path.expanduser('~/data/sgpmetE13b1/')

Create and sort a list of files.


In [ ]:
all_files = os.listdir(in_directory)
all_files.sort()

Now we append the path to each file. 

In [ ]:
fqn_files = [os.path.join(in_directory,thisfile) for thisfile in all_files]

In [ ]:
test_file_name = os.path.join(in_directory, all_files[23])
test_dataset = xr.open_dataset(test_file_name)

X-Array is awesome. It is very rich and couples very nicely to ARM data. 

In [ ]:
test_dataset.time[0].values

Lets plot one day's worth of data


In [ ]:
my_figure = plt.figure(figsize=[10,5])
test_dataset.temp_mean.plot()

So the specific application we want: What is the min, mean and max temperature for a day? (of course this is a UTC day)

In [ ]:
my_mean = float(test_dataset.temp_mean.mean())
my_max = float(test_dataset.temp_mean.max())
my_min = float(test_dataset.temp_mean.min())

In [ ]:
print("Mean: {}, Max: {}, Min: {}".format(my_mean, my_max, my_min ))

Good housekeeping, close the file (this gets you into all kind of trouble when doing distributed computing)

In [ ]:
test_dataset.close()

## So, here is your Distrubuted computing 101. 
To use something like Dask you want to break your problem down into repeated tasks. Then instead of looping you do a *Map Reduce*. Create a function that you want to map onto a list of "things". In our case it is a function that opens a data file, returns the time, mean, min and max.

In [ ]:
def analyze_this_days_temps(filename):
    this_dataset = xr.open_dataset(filename)
    my_mean = float(this_dataset.temp_mean.mean())
    my_max = float(this_dataset.temp_mean.max())
    my_min = float(this_dataset.temp_mean.min())
    this_dataset.close()
    return this_dataset.time[0].values, my_mean, my_max, my_min

test our function on one file

In [ ]:
analyze_this_days_temps(fqn_files[300])

In [ ]:
client

**Here is the magic!** Map the list of files to the function...

In [ ]:
future = client.map(analyze_this_days_temps, fqn_files)

**Show me the PROGRESS!**


In [ ]:
progress(future)

Once it is done gather the data from the complete processes. (Map, then reduce) 

In [ ]:
my_data = client.gather(future)

In [ ]:
times = np.array([tpl[0] for tpl in my_data])
means = np.array([tpl[1] for tpl in my_data])
maxs = np.array([tpl[2] for tpl in my_data])
mins = np.array([tpl[3] for tpl in my_data])

In [ ]:
my_fig = plt.figure(figsize=[10,5])
plt.plot(times, means, '-k', label='Mean')
plt.plot(times, mins, '-b', label='Min')
plt.plot(times, maxs, '-r', label='Max')
plt.ylabel('Temperature (C)')
plt.xlabel('Time')
plt.legend()

